In [1]:
import scipy
import random
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil
from sklearn.decomposition import PCA

In [2]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()

beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

A_full = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Oct 27 05:32:03 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Oct 27 05:32:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 27 05:32:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 27 05:32:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation       

(CVXPY) Oct 27 05:32:11 PM: 21  3.3e-07  1.1e-06  3.1e-11  1.00e+00   4.204653231e+01   4.204653206e+01   2.3e-09  4.14  
(CVXPY) Oct 27 05:32:12 PM: 22  9.5e-08  3.1e-07  4.8e-12  9.56e-01   4.204568395e+01   4.204568387e+01   6.6e-10  4.31  
(CVXPY) Oct 27 05:32:12 PM: 23  7.2e-08  7.8e-08  6.2e-13  1.01e+00   4.204544808e+01   4.204544806e+01   1.7e-10  4.42  
(CVXPY) Oct 27 05:32:12 PM: Optimizer terminated. Time: 4.53    
(CVXPY) Oct 27 05:32:12 PM: 
(CVXPY) Oct 27 05:32:12 PM: 
(CVXPY) Oct 27 05:32:12 PM: Interior-point solution summary
(CVXPY) Oct 27 05:32:12 PM:   Problem status  : PRIMAL_AND_DUAL_FEASIBLE
(CVXPY) Oct 27 05:32:12 PM:   Solution status : OPTIMAL
(CVXPY) Oct 27 05:32:12 PM:   Primal.  obj: 4.2045448085e+01    nrm: 1e+02    Viol.  con: 6e-06    var: 2e-06    cones: 0e+00  
(CVXPY) Oct 27 05:32:12 PM:   Dual.    obj: 4.2045448063e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 8e-07    cones: 0e+00  
------------------------------------------------------------------

In [7]:
import numpy as np
from scipy.optimize import linprog

start_time = time.time()

c = np.ones(A_full.shape[1])  # تابع هدف معمولاً یک بردار تمام یک‌ها است
  
result = linprog(c, A_eq=A_full, b_eq=dose_full_1d, bounds=(0, None))
 
x_opt = result.x
print("بردار بهینه x:", x_opt)

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرای نرم یک ماتریس  \n: {total_execution_time} ثانیه")




بردار بهینه x: None
زمان کل اجرای نرم یک ماتریس  
: 468.2497580051422 ثانیه


In [ ]:
import numpy as np
from scipy.optimize import linprog

start_time = time.time()

c = np.ones(A_full.shape[1])  # تابع هدف معمولاً یک بردار تمام یک‌ها است
  
result = linprog(c, A_eq=A_full, b_eq=dose_full_1d, bounds=(0, None))
 
x_opt = result.x
print("بردار بهینه x:", x_opt)

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرای نرم یک ماتریس  \n: {total_execution_time} ثانیه")


